In [1]:
from pathlib import Path
import numpy as np

from pydrake.geometry.all import MeshcatVisualizer

from pydrake.multibody.plant import (
    AddMultibodyPlantSceneGraph,
)
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import Context, DiagramBuilder
from pydrake.geometry import Rgba, Role, SceneGraph, StartMeshcat


#### Set the working directory to the repo source

In [2]:
import os
os.chdir(Path("../"))

#### Set up Multibodyplant etc

In [3]:
meshcat = StartMeshcat()

builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)

visualizer = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)
diagram = builder.Build()

INFO:drake:Meshcat listening for connections at http://localhost:7002


#### Load trajectory file

In [4]:
from humanoid_footstep_ik.visualizer import FootstepTrajectory


# Put the trajectory file in the repo source folder in a folder named 'data
datapath = Path("data/example_data.pkl")
traj = FootstepTrajectory.load(datapath)

# TODO: The current height doesn't work with IK
traj.com_z = 0.8

Define which indices we will visualize atlas for

In [5]:
num_atlas_frames = 3
# NOTE: Here we evenly space out the frames based on the number above.
# Replace this logic with manual frames if desired!

# Pick out the indices to visualize, so that we start with the first and end with the last,
# and add as many as needed in between to reach viz_params.num_atlas_frames in total.
step_length = int(np.floor(len(traj) / (num_atlas_frames - 1)))
indices_to_visualize = np.arange(0, len(traj), step_length)
if indices_to_visualize[-1] != len(traj) - 1:
    indices_to_visualize = np.concatenate([indices_to_visualize, [len(traj) - 1]])

indices_to_visualize = indices_to_visualize.tolist()

Create visualization objects in meshcat (this must be done BEFORE calling plant.Finalize())

In [6]:
from humanoid_footstep_ik.visualizer import create_atlas_visualizations, create_footstep_visualizations


stone_height = 0.8
for stone in traj.stones:
    stone.add_to_plant(plant, stone_height)

viz_atlases = create_atlas_visualizations(
    plant, indices_to_visualize
)

viz_feet = create_footstep_visualizations(plant, traj)

#### If desired, print some info about the Atlas model

In [7]:
debug = False
if debug:
    viz_atlases[0].print_details()

Finalize plan, turn off collision checking, and create context

In [8]:
plant.Finalize()

# turn off collision checking (there will be collisions between atlases and feet etc.)
turn_off_collision_checking = True
if turn_off_collision_checking:
    source_id = plant.get_source_id()
    for geometry_id in scene_graph.model_inspector().GetAllGeometryIds():
        scene_graph.RemoveRole(source_id, geometry_id, Role.kProximity)


context = diagram.CreateDefaultContext()
plant_context = plant.GetMyMutableContextFromRoot(context)

simulator = Simulator(diagram, context)
simulator.set_target_realtime_rate(1.0)
time = 0.0

#### Update positions
This code block can be rerun.

In [9]:
from humanoid_footstep_ik.visualizer import set_atlas_positions, set_feet_positions

robot_z_rotation = -np.pi/2

set_feet_positions(plant_context, traj, viz_feet, robot_z_rotation)
# NOTE: With the current plan, IK is not solving if we also enforce com position
set_atlas_positions(
    plant_context, traj, viz_atlases, indices_to_visualize, robot_z_rotation, force_com=False
)

# Advance the simulator to update the positions etc (to make it possible to rerun this cell)
simulator.AdvanceTo(time)
time += 0.01